<a href="https://colab.research.google.com/github/meblo98/AT1-2024/blob/main/D%C3%A9s%C3%A9quilibr%C3%A9es_et_Analyse_par_Odds_Ratio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cellule 1: Importation des bibliothèques nécessaires
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from xgboost import XGBClassifier
from sklearn.svm import SVC
import time
import zipfile
import os
import warnings
warnings.filterwarnings('ignore')

# Cellule 2: Chargement et exploration du dataset
# Extraction du fichier ZIP
zip_path = 'depression_dataset_SHAP_PEACE (1).zip'  # À adapter selon le chemin sur votre machine
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('depression_dataset')

# Charger le dataset (suppose un fichier CSV dans le ZIP)
# Remplacez 'dataset.csv' par le nom réel du fichier extrait
data = pd.read_csv('depression_dataset/dataset.csv')  # À adapter

# Exploration initiale
print("Aperçu des données:")
print(data.head())
print("\nColonnes:", data.columns)
print("\nDistribution de la classe cible:")
print(data['target'].value_counts(normalize=True))  # Remplacez 'target' par le nom de la colonne cible

# Supposons que 'target' est la colonne de dépression (0: non-déprimé, 1: déprimé)
X = data.drop('target', axis=1)
y = data['target']

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardisation des données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Cellule 3: Fonction pour évaluer les modèles
def evaluate_model(y_true, y_pred, model_name):
    print(f"\nÉvaluation du modèle: {model_name}")
    print(classification_report(y_true, y_pred))

    # Métriques pour la classe minoritaire (classe 1)
    precision = precision_score(y_true, y_pred, pos_label=1)
    recall = recall_score(y_true, y_pred, pos_label=1)
    f1 = f1_score(y_true, y_pred, pos_label=1)

    return {'precision': precision, 'recall': recall, 'f1': f1}

# Cellule 4: Modèle conçu pour les données déséquilibrées (XGBoost avec scale_pos_weight)
# Calcul du scale_pos_weight
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

start_time = time.time()
xgb_model = XGBClassifier(scale_pos_weight=scale_pos_weight, random_state=42)
xgb_model.fit(X_train_scaled, y_train)
xgb_pred = xgb_model.predict(X_test_scaled)
xgb_time = time.time() - start_time

xgb_metrics = evaluate_model(y_test, xgb_pred, "XGBoost avec scale_pos_weight")
print(f"Temps de calcul: {xgb_time:.2f} secondes")

# Cellule 5: Modèle supplémentaire (SVM avec class_weight='balanced')
start_time = time.time()
svm_model = SVC(class_weight='balanced', random_state=42)
svm_model.fit(X_train_scaled, y_train)
svm_pred = svm_model.predict(X_test_scaled)
svm_time = time.time() - start_time

svm_metrics = evaluate_model(y_test, svm_pred, "SVM avec class_weight='balanced'")
print(f"Temps de calcul: {svm_time:.2f} secondes")

# Cellule 6: SMOTE + PCA + Random Forest
start_time = time.time()

# Application de SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

# Application de PCA
pca = PCA(n_components=0.95)  # Conserver 95% de la variance
X_train_smote_pca = pca.fit_transform(X_train_smote)
X_test_pca = pca.transform(X_test_scaled)

# Random Forest
rf_smote = RandomForestClassifier(random_state=42)
rf_smote.fit(X_train_smote_pca, y_train_smote)
rf_smote_pred = rf_smote.predict(X_test_pca)
rf_smote_time = time.time() - start_time

rf_smote_metrics = evaluate_model(y_test, rf_smote_pred, "SMOTE + PCA + Random Forest")
print(f"Temps de calcul: {rf_smote_time:.2f} secondes")

# Cellule 7: Borderline-SMOTE + Random Forest
start_time = time.time()

# Application de Borderline-SMOTE
borderline_smote = BorderlineSMOTE(random_state=42)
X_train_borderline, y_train_borderline = borderline_smote.fit_resample(X_train_scaled, y_train)

# Random Forest
rf_borderline = RandomForestClassifier(random_state=42)
rf_borderline.fit(X_train_borderline, y_train_borderline)
rf_borderline_pred = rf_borderline.predict(X_test_scaled)
rf_borderline_time = time.time() - start_time

rf_borderline_metrics = evaluate_model(y_test, rf_borderline_pred, "Borderline-SMOTE + Random Forest")
print(f"Temps de calcul: {rf_borderline_time:.2f} secondes")

# Cellule 8: Comparaison des résultats
results = pd.DataFrame({
    'Modèle': ['XGBoost', 'SVM', 'SMOTE + PCA + RF', 'Borderline-SMOTE + RF'],
    'Precision': [xgb_metrics['precision'], svm_metrics['precision'], rf_smote_metrics['precision'], rf_borderline_metrics['precision']],
    'Recall': [xgb_metrics['recall'], svm_metrics['recall'], rf_smote_metrics['recall'], rf_borderline_metrics['recall']],
    'F1-Score': [xgb_metrics['f1'], svm_metrics['f1'], rf_smote_metrics['f1'], rf_borderline_metrics['f1']],
    'Temps (s)': [xgb_time, svm_time, rf_smote_time, rf_borderline_time]
})

print("\nComparaison des performances sur la classe minoritaire:")
print(results)

# Visualisation
plt.figure(figsize=(10, 6))
results.set_index('Modèle')[['Precision', 'Recall', 'F1-Score']].plot(kind='bar')
plt.title('Comparaison des métriques pour la classe minoritaire')
plt.ylabel('Score')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Cellule 9: Analyse des critères de comparaison
print("\nAnalyse des critères:")
print("1. Précision, Rappel, F1-Score: Voir tableau ci-dessus")
print("2. Robustesse: XGBoost et Random Forest sont robustes pour les données déséquilibrées")
print("3. Temps de calcul: XGBoost est rapide, SVM plus lent, SMOTE ajoute du temps")
print("4. Facilité de mise en œuvre: XGBoost et Random Forest sont simples à configurer")
print("5. Capacité à traiter le déséquilibre: XGBoost et SVM gèrent bien sans rééchantillonnage")

print("\nRéflexions:")
print("- Le rééchantillonnage n'est pas toujours nécessaire si les modèles pondèrent les classes.")
print("- Compromis: XGBoost peut favoriser la classe minoritaire au détriment de la précision globale.")
print("- Les pondérations de classe suffisent pour des déséquilibres modérés.")

# Cellule 10: Analyse des Odds Ratios
# Supposons une variable 'PHQ9_score' (score de dépression) pour l'analyse
# À adapter selon l'image et les variables du dataset
print("\nAnalyse des Odds Ratios:")

# Hypothèse basée sur une visualisation supposée
print("Hypothèse: Les scores PHQ-9 élevés sont fortement associés à la dépression.")

# Création d'une variable binaire pour 'PHQ9_score' (au-dessus/en-dessous de la médiane)
data['High_PHQ9'] = (data['PHQ9_score'] > data['PHQ9_score'].median()).astype(int)  # À adapter

# Calcul des odds ratios
def calculate_odds_ratio(data, feature, target):
    contingency_table = pd.crosstab(data[feature], data[target])
    odds_ratio = (contingency_table.iloc[1,1] * contingency_table.iloc[0,0]) / \
                 (contingency_table.iloc[1,0] * contingency_table.iloc[0,1])
    return odds_ratio

or_phq9 = calculate_odds_ratio(data, 'High_PHQ9', 'target')
print(f"Odds Ratio pour High_PHQ9: {or_phq9:.2f}")

# Interprétation
print("\nInterprétation:")
if or_phq9 > 1:
    print("L'OR suggère que des scores PHQ-9 élevés sont associés à la dépression.")
else:
    print("L'OR suggère que des scores PHQ-9 élevés ne sont pas fortement associés à la dépression.")
print("Si la visualisation montrait une forte corrélation visuelle mais un OR faible, cela pourrait indiquer un effet de confusion.")
print("Vérifiez les interactions avec d'autres variables (ex. : âge, genre).")

# Cellule 11: Conclusion
print("\nConclusion:")
print("XGBoost avec scale_pos_weight est efficace pour les données déséquilibrées sans rééchantillonnage.")
print("Les odds ratios peuvent révéler des contradictions dues à des effets de confusion.")
print("Pour des données très déséquilibrées, combiner pondération et rééchantillonnage peut être optimal.")

FileNotFoundError: [Errno 2] No such file or directory: 'depression_dataset_SHAP_PEACE (1).zip'